# Focusstacking in PYNQ on Xilinx Kria KV260
***
Version: V1.1 as of 29.03.2022
Author: Andreas Rudolph as https://github.com/produkt-manager/focusstacking

Helper documentation
- Markdown cheat sheet https://www.ibm.com/docs/en/watson-studio-local/1.2.3?topic=notebooks-markdown-jupyter-cheatsheet
- 2019 (current version) https://www.xilinx.com/content/dam/xilinx/support/documentation/sw_manuals/xilinx2019_1/ug1233-xilinx-opencv-user-guide.pdf
- This describes how to work with user defined functions https://notebook.community/evanmiltenburg/python-for-text-analysis/Chapters/Chapter%2011%20-%20Functions%20and%20scope


This notebook is inspired by/ reusing https://github.com/cmcguinness/focusstack (licensed as by https://www.apache.org/licenses/LICENSE-2.0)

## 1. How To use this Notebook
***
This Notebook uses PYNQ on a KV260, and employs OpenCV in order to implement its imaging features. Perform these steps to run the notebook, after you have uploaded it to your board:

1. Use your own focal stack, or download images from Wikipedia https://de.wikipedia.org/wiki/Focus_stacking
2. Create the following folders in the PYNQ file system on your XILINX KRIA KV260 in order to be able to store the images in the file system
    - "inbound_images" 
    - "outbound_images"
    - "test_images"
    - "temp_images"

3. Store the images to stack in the folder __"inbound_images"__. The first image of your images to stack will serve as a reference image. All other images will be the stack and they relate to this reference.
4. Run the notebook, and alter the parameters to fit your needs
5. The notebook produces a stacked image, and stores it in the folder __"inbound_images"__

## 2. Definitions and Preparations
***
Import libraries, set all paths, and initialize variables

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
import glob, os
import cv2

### 2.1. Set Debug Mode

Set this switch to true, if you want to have a log of actions

In [ ]:
debugging = True
# debugging = False

### 2.2. Set Path to Images

Define where to find the images in the file system of your installation

In [ ]:
path2inimages = "inbound_images"  # Input of raw-Images to be stacked 
path2outimages = "outbound_images" # Output of stacked image
path2testimages = "test_images" # Test images
path2tempimages = "temp_images" # Intertemporal results

delimiter = "/"

### 2.3. Settings for the Perspective Transformation Process in Image Alignment

The homography, and it's corresponding function in OpenCV represents a perspective transformation between two planes. This function is needed to align the different images of a focus stack to the reference image.

The process is defined as following (see https://en.wikipedia.org/wiki/Homography_(computer_vision):

> _"In the field of computer vision, any two images of the same planar surface in space are related by a homography (assuming a pinhole camera model). This has many practical applications, such as image rectification, image registration, or camera motion—rotation and translation—between two images. Once camera resectioning has been done from an estimated homography matrix, this information may be used for navigation, or to insert models of 3D objects into an image or video, so that they are rendered with the correct perspective and appear to have been part of the original scene (see Augmented reality)."_ 

OpenCV's homography method is defined as

> _ "openCV cv.findHomography(srcPoints, dstPoints[, method[, ransacReprojThreshold[, mask[, maxIters[, confidence]]]]])

and it returns a returnvalue __"retval'__, as well as a __"mask"__.

In this method OpenCV offers different calculations for the homography. Set the corresponding switch in order to define, which method is used in this notebook. 

> _ Methods 0 - a regular method using all the points, i.e., the least squares method

> _ RANSAC - RANSAC-based robust method

> _ LMEDS - Least-Median robust method

> _ RHO - PROSAC-based robust method

The parameter __"ransacReprojThreshold"__ defines, the threshold as of which a point i is considered as an outlier. The sourcepoints __"srcPoints"__ and the destination points __"dstPoints"__ are measured in pixels. Usually these parameters are set in a range of 1 to 10.

In [ ]:
# Define the method to use. To do so, set one parameter to true, and the rest to false

setRANSAC = True # RANSAC - RANSAC-based robust method
setLMEDS = False # LMEDS - Least-Median robust method
setRHO = False # RHO - PROSAC-based robust method

### 2.4. Settings for the Image Alignment Method

As you can use different methods for image alignment you can set here the corresponding switches _(Remember that gosting in your output image might indicate that images are missaligned, and this might be the result of either the method, or the process parameters.)_

* SIFT (Scale-Invariant Feature Transform), as described here https://docs.opencv.org/4.x/da/df5/tutorial_py_sift_intro.html. This method goes back to __"D.Lowe, University of British Columbia (2004)"__, who came up with a new algorithm that is able to extract keypoints and compute its descriptors, as needed by image scaling transactions. This algorithm is covered by patents.

* ORB (Oriented FAST and Rotated BRIEF) keypoint detector and descriptor extractor as described here https://docs.opencv.org/3.4/d1/d89/tutorial_py_orb.html. ORB was created by Rublee, et.al in their 2011'er paper __"ORB: An efficient alternative to SIFT or SURF"__. The algorithm is an alternative to SIFT and SURF in terms of computation cost and matching performance. While SIFT and SURF are covered by patents, ORB is open Source.

In [ ]:
#   Set one alignment method to true, the rest to false
setSIFT = True
setORB = False

### 2.5. Settings for the LoB Filter (Laplacian Blur)

Focus stacking requires that those parts of an image are identified, that appear to be in-focus, while out-of-focus areas are neglected. 

The parameters for the __"size of laplacian kernel window"__, and the __"size of the kernal that is used for the gaussian blur"__ allow you to influence, which areas are considered as sharp in an image. You can alter and tune these parameters, so that they match your needs. 

In order to obtain good results,  these parameters typically should be set to identical in value, or they should at least fall into the same range of values.

Due to the functional principles of the involved algorithms, make sure that you only choose odd numers.

In [ ]:
# Switches for the LoB filter

kernel_size = 5         # Size of the laplacian kernel window
blur_size = 5           # Size of the kernal that is used for the gaussian blur

## 3. Import and Show the Image Objects in Frontend
***
The current version of the notebook is assuming that user takes a stack of images, and places them in the import image folder. In the current version, fotos of format <> JPG, or PNG are not processed, and therefore removed before the import without further notice.

This section reads the images from the file system into one list. This notebook is assuming, that all photos found within this folder need to be stacked. Photo [0] will thereby be considered as the reference-image. 

In a future version of this notebook, the images-to-be-stacked might originate from a camera module that is attached to the board, and a focus rail that is driven by the same board. For such tasks the Composable Pipeline Overlay in the standard PYNQ installation is a good starting point.

### 3.1. Read the Image Names from the Import Folder

Create a list of images to be read/ imported.

In [ ]:
# Read image names into a list and skip files in the wrong format

inbound_image_names  = sorted(os.listdir(path2inimages))

if debugging:  
    print ("Name of files stored in image path {}".format(inbound_image_names))

# Test for wrong formats and remove from filelist
for inbound_image_name in inbound_image_names:
    if inbound_image_name.split(".")[-1].lower() not in ["jpg", "jpeg", "png"]:
        if debugging:  
            print ("Remove a file that has a wrong format {}".format(inbound_image_name))
        inbound_image_names.remove(inbound_image_name)

# Alternative
# file, ext = os.path.splittext(inbound_image_name)
# if ext not in ["jpg", "jpeg", "png"]:
#        if debugging:  
#            print ("Remove file with wrong format {}".format(inbound_image_names))
#        inbound_image_names.remove(inbound_image_name)

### 3.2. Read images into the List "original_images"

Import images from file system, and create thumbnails.

The create thumbnails step is optional and it is helpful, in order to make sure that images are available for presentation purposes that are smaller in size. Alternatively you can use image pyramids. This describes how to create an image pyramid: 
https://docs.opencv.org/3.4/d4/d1f/tutorial_pyramids.html

In [ ]:
# Read the images into a list of original_images

original_images = [] # images
original_images_thumb = [] # thumbnails

for inbound_image_name in inbound_image_names:
    if debugging:
        print ("Reading original image with name {}".format(inbound_image_name))

# OpenCV-read one photo, output size, and append to the list of original_images
    one_original_image = cv2.imread(path2inimages + delimiter + "{}".format(inbound_image_name))
    if debugging:
        one_original_image_width, one_original_image_height = one_original_image.size
        print("Original image with size: {}x{} pixels.".format(one_original_image_width, one_original_image_height))
    original_images.append(one_original_image)

    # Create thumbnails and store in separate path
    tn_size = 128, 128
    one_original_image_thumb = Image.fromarray(np.asarray(one_original_image))
    one_original_image_thumb.thumbnail(tn_size)
    if debugging:
        one_original_image_thumb_width, one_original_image_thumb_height = one_original_image_thumb.size
        print("Original thumbnail with size: {}x{} pixels.".format(one_original_image_thumb_width, one_original_image_thumb_height))
    one_original_image_thumb.save(path2tempimages + delimiter  + "thumbnail" + "{}".format(inbound_image_name), "JPEG")   
    original_images_thumb.append(one_original_image_thumb)

if debugging:
    print ("Number of images read from folder original_images {}".format(len(original_images)))
    print ("Number of images converted into thumbnails {}".format(len(original_images_thumb)))


### 3.3. Create a Copy of List "images_2_stack"

Copy the list named __"original_images"__ to a list named __"images_2_stack"__. __"Images_2_stack"__ will lateron be processed with focus stacking, and this step allows you to run a before-after comparision.

In [ ]:
images_2_stack = original_images.copy()

if debugging:
    print ("Number images in images_2_stack list {}".format(len(images_2_stack)))

### 3.4. Display Images in Notebook Frontend

After we have imported all images, we now show all images in the frontend. For this task we use the standard pillow library and therefore it is necessary to color convert images between the color format for OpenCV, and for Pillow.

As you can verify from the output, the list __"original_images"__ includes a stack of images with varying focus planes. Observe how the focus plane shifts from image to image.

In [ ]:
canvas = plt.gcf()
size = canvas.get_size_inches()
canvas.set_size_inches(size*2)

# axis off
plt.axis("off")

# create grid
concat_original_images = np.concatenate(original_images, axis=1)
if debugging:
    print ("Number of images in OpenCV color space {}".format(len(original_images)))
# wird überschrieben ---> anpassen 
    plt.imshow(concat_original_images)

# Color Convert, as OpenCV imread() has order of colors BGR (blue, green, red), but Pillow assumes RGB. 
concat_original_images_RGB = cv2.cvtColor(concat_original_images, cv2.COLOR_BGR2RGB)
plt.imshow(concat_original_images_RGB)

## 4. Focus Stacking of imported Images
***

After having imported the images, they can now be stacked. Basic idea/ algorithm: From the different images identify the points that are in focus and merge theses points into a common image.

These are the high level processing steps covered in this part of the notebook:
* Align the individual images that define the stack (thereby one image saves as reference)
* Apply a lapacian blur filter. This filter is used to identify the sharpest points in an image
* Create the common image from the single images of the stack, by using the absolute value of the LoB filter as a criterion 

### 4.1. Align Images that belong to the Focal Stack

Due to micromovements of the camera inbetween shots, or due to a moving object that is captured, it is possible that the individual images of the stack are displaced. Therefore it is first necessary to align all images to one reference image, and to make sure that they overlap properly.

Wrongly aligned images may lead to ghosting in the final stacked image.

#### 4.1.1. Define the Detector Functions

Preparation of image alignment, either using the SIFT or the ORB process as described above.

##### SIFT

This notebook uses the Scale Invariant Feature Transform (SIFT) class for extracting keypoints and computing descriptors as described above. The creation method is defined as

> _"cv.SIFT_create([, nfeatures[, nOctaveLayers[, contrastThreshold[, edgeThreshold[, sigma]]]]])"_ 

According to the documentation, these are the parameters:

> _- "nfeatures: The number of best features to retain. The features are ranked by their scores (measured in SIFT algorithm as the local contrast)"_

> _- "nOctaveLayers: The number of layers in each octave. 3 is the value used in D. Lowe paper. The number of octaves is computed automatically from the image resolution."_

> _- "contrastThreshold:	The contrast threshold used to filter out weak features in semi-uniform (low-contrast) regions. The larger the threshold, the less features are produced by the detector."_

##### ORB

The class that is implementing the ORB keypoint detector and descriptor extractor, is described here https://docs.opencv.org/3.4/db/d95/classcv_1_1ORB.html

According to the documentation, the algorithm detects stable keypoints by means of using FAST in pyramids. Then it selects the strongest features either using FAST or using the Harris response. It finds the orientation of the strongest features using first-order moments. The algorithm then computes the descriptors using a method in which the coordinates of random point pairs (or k-tuples) are rotated according to the measured orientation.

In [ ]:
aligned_images_2_stack = []

if setSIFT:
    detector = cv2.xfeatures2d.SIFT_create()
elif setORB:
    detector = cv2.ORB_create(1000)
else:
    if debugging:
        print ("ERROR with setting either the ORB or the SIFT detector - should not happen")

#### 4.1.2. Detect Keypoints and Compute the Descriptors for the Reference Image

In order to align the images-to-be-stacked, we identify keypoints in each image. Afterwards we move the images in a way that these keypoints are located at the same location. Thereby the first image in the stack acts as the "base" or reference image.

The reference image is prepared as follows:

* OpenCV offers the __"DetectAndCompute"__ method of the detector class for the keypoint-detection-task. This method in particular detects keypoints and computes their descriptors. It returns the keypoints and the descriptors of the image, and is defined as __"cv.Feature2D.detectAndCompute(	image, mask[, descriptors[, useProvidedKeypoints]])-> keypoints, descriptors"__

* For best results, we convert the color image to greyscale using OpenCV's function __"cvtColor"__ that accepts the image to be converted, and a setting that defines that we will convert the colors BGR (blue, green, red) to grey (*Remark: OpenCV uses the colors BGR (blue, green, red) color order, while Pillow assumes RGB*)


In [ ]:
# Detect keypoints and compute the descriptors

if debugging:
    print ("Detecting features of base image that can ten be used for alignment")

aligned_images_2_stack.append(images_2_stack[0])  # Reference image
reference_image_in_gray = cv2.cvtColor(images_2_stack[0],cv2.COLOR_BGR2GRAY)
reference_image_kp, reference_image_desc = detector.detectAndCompute(reference_image_in_gray, None)

#### 4.1.3. Align Images by Means a Brute-Force Descriptor Matcher, Homography-Finder and Image Warping 

The alignment process consists of the following steps
* Detect keypoints
* Using a brute-force descriptor matcher the topmost matches for the keypoints of the image is returned, and the matches are paired.
* Using a homography algorithmn the matrix of of aligned image points is calculared
* Using these matrices, the images are warped so that the rlevant keypoints are located at identical positions


##### A) Key Point Detection

Using the same __"detectAndCompute"__ method as described above, each of the non-reference images in the list of images-to-be-stacked is converted.

##### B) Brute Force Matcher

Using the resulting matrix of identified keypoints, a brute-force descriptor matcher is used to idenfify matching points within the images (reference image, and image-in-process). As mentioned above, OpenCV offers different Brute-force descriptor matcher methods, depending on the fact, which processing method is used (SIFT or ORB).

* __Brute-force descriptor matcher for SIFT__: For each descriptor in the first set, this matcher finds the closest 
descriptor in the second set by trying each descriptor. It is possible to mask permissible matches of descriptor sets, and make sure that the methid applies to selected areas only. 

* __Brute-force descriptor matcher for ORB__: The matcher is called with two sets. For each descriptor in the first set, it finds the closest descriptor in the second set. He does so, by trying each one. This descriptor matcher as well supports masking. The matcher is as well called with a normtype, which defines the calculation method.

The create method of the BFMatcher sets the norm type, and a crosscheck-flag. Both define, how the matcher behaves. The optimal normtype depends on the used method. The documentation defines the optimal use as this:

> "normType: One of NORM_L1, NORM_L2, NORM_HAMMING, NORM_HAMMING2. L1 and L2 norms are preferable choices for SIFT and SURF descriptors,NORM_HAMMING should be used with ORB, BRISK and BRIEF, NORM_HAMMING2 should be used with ORB when WTA_K==3 or 4."

>"crossCheck: If it is false, this is will be default BFMatcher behaviour when it finds the k nearest neighbors for each query descriptor. If crossCheck==true, then the knnMatch() method with k=1 will only return pairs (i,j) such that for i-th query descriptor the j-th descriptor in the matcher's collection is the nearest and vice versa, i.e. the BFMatcher will only return consistent pairs. Such technique usually produces best results with minimal number of outliers when there are enough matches. This is alternative to the ratio test, used by D. Lowe in SIFT paper."

The current notebook uses the Hamming norm. If the __NORM_HAMMING__ parameter is used with one input array, it calculates the Hamming distance of the array from zero. If this parameter is used with two input arrays, the Hamming distance between these arrays is calculated.


##### C) Pair Matches

After matches are found with the brute-force descriptor matcher, the __"knnMatch"__ method is used to pair these matches, and to identify the two topmost matches. Only those pairs below a certain threshold are returned.

This returns the top two matches for each feature point (list of list)

The method is defined as __"cv.DescriptorMatcher.knnMatch(queryDescriptors, trainDescriptors, k[, mask[, compactResult]]) ->matches"__, while k defines the number of best matches that is found with each query descriptor.

##### D) Homography
As described above, OpenCV's homography finder, finds the information that is needed to apply a perspective transformation between two planes. This function allows us to align the different images of a focus stack to the reference image.

* reference_image_points -> Keypoints from reference photo
* stacked_images_points -> Keypoints from photos to stack
* matched points (Best match for each descriptor from a query set)

##### E) Warp Image

After Homography is calculated, the images are warped and undergo a perspective transformation.

The notebook uses the __warpPerspective()__ method in order to apply a perspective transformation to each image (see documentation here https://docs.opencv.org/3.4/da/d54/group__imgproc__transform.html#gaf73673a7e8e18ec6963e3774e6a94b87)

To do so, when the flag WARP_INVERSE_MAP is set, the __warpPerspective__ function transforms the source image using the specified matrix. If the parameter is not set, the transformation is first inverted with invert and then put in the formula above instead of M. The function cannot operate in-place.

In [ ]:

# Image Alignment main loop

for i in range(1,len(images_2_stack)):
    if debugging:
        print ("Aligning image {}".format(i))
    # Keypoint detection
    stacked_images_kp, stack_image_desc = detector.detectAndCompute(images_2_stack[i], None)

    # Brute-force descriptor matching
    if setSIFT:        
        bf = cv2.BFMatcher()
        pairMatches = bf.knnMatch(stack_image_desc,reference_image_desc, k=2)

        # Select only certain matches by distance
        rawMatches = []
        for m,n in pairMatches:
            if m.distance < 0.7*n.distance:
                rawMatches.append(m)
    elif setORB:
        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

        #  match cv.DescriptorMatcher.match(	queryDescriptors, trainDescriptors[, mask]	) ->	matches. 
        # Find the best match for each descriptor from a query set.
        rawMatches = bf.match(stack_image_desc, reference_image_desc)

    # Final processing for the matches (sort rawMatches array by distance)
    sortMatches = sorted(rawMatches, key=lambda x: x.distance)
    matches = sortMatches[0:128]

   # -----> H O M O G R A P H Y 

    # Initialize the needed arrays
    stacked_images_points = np.zeros((len(matches), 1, 2), dtype=np.float32)
    reference_image_points = np.zeros((len(matches), 1, 2), dtype=np.float32)

    # As example shows: np.zeros((5,), dtype=int) -> array([0, 0, 0, 0, 0]) --> array filled with zeros of datatype int
    for i in range(0,len(matches)):
        stacked_images_points[i] = stacked_images_kp[matches[i].queryIdx].pt
        reference_image_points[i] = reference_image_kp[matches[i].trainIdx].pt

    # Selected method (see above settings)
    if setRANSAC:
        the_homography, mask = cv2.findHomography(stacked_images_points, reference_image_points, cv2.RANSAC, ransacReprojThreshold=2.0)
    elif setLMEDS:
        # ransacReprojThreshold does not work with LMEDS
        the_homography, mask = cv2.findHomography(stacked_images_points, reference_image_points, cv2.LMEDS)
    elif setRHO:
        the_homography, mask = cv2.findHomography(stacked_images_points, reference_image_points, cv2.RHO, ransacReprojThreshold=2.0)
    else:
        print ("ERROR with setting homography method")

    # Warp Images to match the points found in homography
    warpedimage = cv2.warpPerspective(images_2_stack[i], the_homography, (images_2_stack[i].shape[1], images_2_stack[i].shape[0]), flags=cv2.INTER_LINEAR)

    # Store warped image in result list   
    aligned_images_2_stack.append(warpedimage)

### 4.2 Compute the Gradient Map of the aligned Images

Once the images-to-be-stacked are aligned to each other, the gradient maps are calculated. Gaussian blur is added so that pixels in the same focus plane appear to be lighted identically.

Thereby the calculated blurred laplacian kernel serves as a measure for the parts of the image that are in focus. The Laplacian of Gaussian (LoG) filter is defined as the Laplace operator applied to a Gaussian kernel. Thereby the LoG is a line detector.

The following document describes the needed theoretical background: 
* https://www.crisluengo.net/archives/1099/
* https://rohanchitnis.com/cs194-26/rohan_chitnis_proj3/index.html

The amount of the laplacian blur filter depends on a parameter that defines the size of the laplacian window (size of the kernel), and it depends on the part of this kernel that will be gaussian blurred. Variations in these parameters might incluence the result.

After the list of images with applied LoB-Filter is available, sufficient information is availalbe to merge those pixels that represent the largest values/ that represent the most sharpest parts of the image.

##### A) Gaussian Blur

The OpenCV function __gaussianBlur__ applies blur to the source image using a Gaussian filter.

The function cuses the specified Gaussian kernel, and kernel size. In-place filtering is as well supported.

For more information, see the documentation https://docs.opencv.org/3.4/d4/d86/group__imgproc__filter.html#gaabe8c836e97159a9193fb0b11ac52cf1


##### B) Laplacian

The OpenCV function __laplacian__ calculates the Laplacian of the source image.

To do so, it either adds the second x and y derivatives, using the Sobel operator (ksize > 1), or it filters the image with a defined 3×3 aperture (ksize == 1). 

For more information, see the documentation https://docs.opencv.org/3.4/d4/d86/group__imgproc__filter.html#gad78703e4c8fe703d479c1860d76429e6

In the current notebook first the Gaussian blur is applied to the image, and then the Laplacian of the blurred image is taken. Therefore this represents the Laplacian Blur filter described above. 

In [ ]:
# Laplacian Blur Filter

if debugging:
    print ("Computing the laplacian of the blurred images - LoB Filter")
  
laplacian_images = []
for i in range(len(aligned_images_2_stack)):
    if debugging:
        print ("Laplacian {}".format(i))
        
    # Image blurring
    a_blurred_image = cv2.GaussianBlur((cv2.cvtColor(aligned_images_2_stack[i],cv2.COLOR_BGR2GRAY), (blur_size,blur_size), 0))

    # Laplacian of the blurred image
    laplacian_images.append(cv2.Laplacian(a_blurred_image, cv2.CV_64F, ksize=kernel_size))
    laplacian_images = np.asarray(laplacian_images)
    if debugging
        print ("Shape of array of laplacians = {}".format(laplacian_images.shape))


### 4.3. Merge the In-Focus-Areas of the Images of the Image Stack

These are the steps to merge the sharpest areas within the individual images-to-be-stacked into the final merged image:

* The absolute values are idenfified. 
* The maxima of these values are taken. 
* A bitwise negation operation is applied in order to collect these largest values.

##### A) Absolute Value

The __"absolute"__ function returns an ndarray containing the absolute value of each element in the input array x. For complex input, a + ib, the absolute value is a scalar if x is a scalar.'

The element-wise absolute value of an array is calculated as shown in this example

x = np.array([-1.2, 1.2])
> np.absolute(x)

array([ 1.2,  1.2])
> np.absolute(1.2 + 1j)

1.5620499351813308

##### B) Maximum

The method __"ndarray.max(axis=None, out=None, keepdims=False, initial=<no value>, where=True)"__ returns the maximum values along a given axis (see also numpy.amax).

##### C) Bitwise not

The  bitwise_not() function converts an input array to an output array, by means of a bitwise negation, while it applies a mask. It therefore inverts every bit of an array, and uses the following parameters __"input array"__, __"output array with the same size and type as the input array"__, __" an optional operation mask, 8-bit single channel array, that specifies elements of the output array to be changed"__

In [ ]:
# Merges images after LoB Filter is applied

# Prepare image array
merged_image = np.zeros(shape=aligned_images_2_stack[0].shape, dtype=aligned_images_2_stack[0].dtype)

# Absolute Values
abs_laplacian_images = np.absolute(laplacian_images)

# Maxima
maxima = abs_laplacian_images.max(axis=0)

# Mask, including type conversion into uint8
bool_mask = abs_laplacian_images == maxima
mask = bool_mask.astype(np.uint8)

# merged_image is the bitwise_not() over the aligned images applying the mask
for i in range(0,len(aligned_images_2_stack)):
    merged_image = cv2.bitwise_not(aligned_images_2_stack[i],merged_image, mask=mask[i])

### 4.4. Store Stacked Image in Filesystem and Show in Frontend

Now store final __"merged_image"__ into folder at output path, and show __"merged_image"__ in frontend.

In [ ]:
# Store images and show it


cv2.imwrite(path2outimages + delimiter + "merged.png".format(inbound_image_name), merged_image)

In [ ]:
canvas = plt.gcf()
size = canvas.get_size_inches()
canvas.set_size_inches(size*2)
plt.axis("off")

# Color Convert, as OpenCV imread() has order of colors BGR (blue, green, red), but Pillow assumes RGB. 
merged_image_RGB = cv2.cvtColor(merged_image, cv2.COLOR_BGR2RGB)
plt.imshow(merged_image_RGB)